# Named Entity Recognition Pipeline

El pipeline toma una URL de un feed en formato RSS, obtiene el título y descripción de los artículos en el feed, detecta las NER con un modelo pre-entrenado, y las muestra ordenadas por frecuencia de aparición.

### Versiones
Probado con:
* Almond 0.6.0
* Ammonite 1.6.7
* Scala library version **2.11.12** -- Copyright 2002-2017, LAMP/EPFL
* Java 1.8.0_282

Para ver más información ir a (Help -> About Scala Kernel)

## 1. Obtener texto

### 1.1 Importar librerías

In [1]:
// Equivalent of adding dependencies to maven or sbt files
// For example, to add "org.scalaj" %% "scalaj-http" % "2.4.2" 
import $ivy.`org.scalaj::scalaj-http:2.4.2`
// "org.scala-lang.modules" %% "scala-xml" % "1.3.0"
import $ivy.`org.scala-lang.modules::scala-xml:1.3.0`

import $ivy.$                              
// "org.scala-lang.modules" %% "scala-xml" % "1.3.0"

import $ivy.$                                        

In [2]:
import scalaj.http.{Http, HttpResponse}
import scala.xml.XML

import scalaj.http.{Http, HttpResponse}

import scala.xml.XML

In [3]:
//IMPORRTAR LIBRERIAS PARA JSON
import $ivy.`org.json4s::json4s-jackson:3.4.0`

import $ivy.$                                 

In [4]:
import org.json4s.JsonDSL._
import org.json4s._
import org.json4s.jackson.JsonMethods._

import org.json4s.JsonDSL._

import org.json4s._

import org.json4s.jackson.JsonMethods._

In [5]:
implicit val formats = DefaultFormats

formats: DefaultFormats.type = org.json4s.DefaultFormats$@6dc9f590

In [6]:
import scala.util.matching.Regex

import scala.util.matching.Regex

### 1.1 Obtener el texto del RSS Feed

Realizamos una consulta HTTP, que nos devuelve una instancia de HTTPResponse. Dentro del atributo `body` de la HTTPResponse, se encuentra el texto del feed en formato XML. Luego, se parsea el XML para extraer los campos `title` y `description`.

In [6]:
// Tutorial https://alvinalexander.com/source-code/scala-how-to-http-download-xml-rss-feed-timeout/
// get the xml content using scalaj-http
val urlRss = "https://www.chicagotribune.com/arcio/rss/category/sports/?query=display_date:[now-2d+TO+now]&sort=display_date:desc"

urlRss: String = "https://www.chicagotribune.com/arcio/rss/category/sports/?query=display_date:[now-2d+TO+now]&sort=display_date:desc"

In [7]:
//url post de reddit
val urlJson = "https://www.reddit.com/r/Android/hot/.json?count=10"

urlJson: String = "https://www.reddit.com/r/Android/hot/.json?count=10"

In [84]:
//url preguntas de Stack Exchange
val urlStack = "https://api.stackexchange.com/2.2/questions?order=desc&sort=activity&site=stackoverflow"

urlStack: String = "https://api.stackexchange.com/2.2/questions?order=desc&sort=activity&site=stackoverflow"

In [8]:
//Para probar que andan las excepciones
val urlTrucho = "Hola"

urlTrucho: String = "Hola"

In [9]:
trait DataSource {
    def getResponse(url: String): String = {
        try{
            val response = Http(url)
            .timeout(connTimeoutMs = 2000, readTimeoutMs = 5000)
            .asString
            
            response.body
        } 
        catch{
            case ex: java.net.MalformedURLException => ""
            case ex: java.net.ConnectException => ""
            case ex: java.net.UnknownHostException => ""
        }
    }
    
    def parseResponse(url: String): Seq[String]
}

defined trait DataSource

In [10]:
class RssSource(fields: Seq[String] = Seq("title", "description")) extends DataSource {
    def parseResponse(url: String): Seq[String] = {
        val xml = XML.loadString(getResponse(url))
        
        (xml\\ "item").map { item =>
            fields.map { field =>
                (item \ s"$field").text + " "
            }.foldLeft("")(_+_)
        }
    }
}

defined class RssSource

In [11]:
class JsonSource(fields: Seq[String] = Seq("title", "selftext")) extends DataSource {        
    //Expresion regular urls
    val urlRegex = "(https?|ftp|file)://[-a-zA-Z0-9+&@#/%?=~_|!:,.;]*[-a-zA-Z0-9+&@#/%=~_|]".r

    def removeUrls(secuencia: Seq[String]): Seq[String] = {
            secuencia.map(urlRegex replaceAllIn(_, " "))
    }

    def parseResponse(url: String): Seq[String] = {
        val res = (parse(getResponse(url))\"data"\"children"\"data").extract[List[Map[String, Any]]]
        
        removeUrls(res.map { Map =>
            fields.map { field =>
                Map(field) + " "
            }.foldLeft("")(_+_)
        })
    }
}

defined class JsonSource

In [89]:
class StackQSource(fields: Seq[String] = Seq("title")) extends DataSource {        
    def parseResponse(url: String): Seq[String] = {
        val res = (parse(getResponse(url))\"items").extract[List[Map[String, Any]]]
        
        res.map { mp =>
            fields.map { field =>
                mp(field) + " "
            }.foldLeft("")(_+_)
        }
    }
}

defined class StackQSource

In [12]:
val rssSource = new RssSource()

rssSource: RssSource = ammonite.$sess.cmd9$Helper$RssSource@433c6b07

In [13]:
val jsonSource = new JsonSource(Seq("title", "selftext"))

jsonSource: JsonSource = ammonite.$sess.cmd10$Helper$JsonSource@280386d5

In [90]:
val stackQSource = new StackQSource()

stackQSource: StackQSource = ammonite.$sess.cmd88$Helper$StackQSource@6a1485d8

In [128]:
val listStack = stackQSource.parseResponse(urlStack)

listStack: Seq[String] = List(
  "Efficient python loop structure to repeat something once ",
  "On html, How to make a type box where the user can submit an answer and I can see all the answers in a text file or something? ",
  "What is the pythonic/idiomatic way of filtering the output of a generator expression? ",
  "Need directions for basics of HPC on MZ Azure ",
  "A google sheet api http request with &quot;Service account key&quot; auth and without google/apiclient ",
  "Using a callback only when a filter returns boolean value true ",
  "SSH2 issues wiht Node.js ",
  "PyKinectv2 Body tracking count ",
  "How do I sum matching strings in a range of ints &amp; display as a stacked bar plot? ",
  "Add time delay in each rows ",
  "Can&#39;t Output Seaborn Scatterplot to Tkinter Frame ",
  "Is it possible to add a &quot;.&quot; inside a Math.random ",
  "Adding voices in the voice list in Festival Software ",
  "How to package a python project that has dependency on tflite_runtime?

In [14]:
val listRss = rssSource.parseResponse(urlRss)

listRss: Seq[String] = List(
  "4 things we heard from Chicago Bears offensive assistants, including competition for Anthony Miller and high expectations for Germain Ifedi Chicago Bears offensive coaches talked last week about some key topics involving veteran players, including wide receivers Anthony Miller and Allen Robinson, tight end Jimmy Graham and right tackle Germain Ifedi. ",
  "When it comes to Justin Fields\u2019 development, \u2018time is the biggest question\u2019 for the Chicago Bears. A look back at past rookie QBs lends insight into the process. Conventional thinking is the Chicago Bears will want to begin the season with Andy Dalton at QB, as the last thing they want to do is risk damaging Justin Fields\u2019 development by throwing him in before he\u2019s ready. But he\u2019s also the shiny new toy in the building, so Matt Nagy and his staff will be tempted to play the rookie. ",
  "Chicago Cubs welcome Jon Lester and Kyle Schwarber back to Wrigley Field \u2014 and th

In [15]:
val listJson = jsonSource.parseResponse(urlJson)

listJson: Seq[String] = List(
  """Sunday Rant/Rage (May 16 2021) - Your weekly complaint thread! Note 1. Join our IRC, and Telegram chat-rooms! [Please see our wiki for instructions.]( )

This weekly Sunday thread is for you to let off some steam and speak out about whatever complaint you might have about:  

* Your device.  

* Your carrier.  

* Your device's manufacturer.  

* An app  

* Any other company

***  

**Rules**  

1) Please do not target any individuals or try to name/shame any individual. If you hate Google/Samsung/HTC etc. for one thing that is fine, but do not be rude to an individual app developer.

2) If you have a suggestion to solve another user's issue, please leave a comment but be sure it's constructive! We do not want any flame-wars.  

3) Be respectful of other's opinions. Even if you feel that somebody is "wrong" you don't have to go out of your way to prove them wrong. Disagree politely, and move on. """,
  """/r/Android Community Feedback Poll (February 

## 2. Detectar las entidades nombradas

### 2.1 Crear el modelo

El **modelo** es sólo la función `getNEs`, que recibe una lista de textos.
Para cada texto, se separa las palabras del texto usando los espacios, y considera que es una entidad nombrada si empieza con mayúscula.

In [161]:
case class NERCount(ner: String, count:Double)

class NERModel(){
    // Este código lista los signos de puntuación y algunas palabras comunes del inglés que se van a sacar del texto.
    val STOPWORDS = Seq (
        "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you",
        "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she",
        "her", "hers", "herself", "it", "its", "itself", "they", "them", "your",
        "their", "theirs", "themselves", "what", "which", "who", "whom",
        "this", "that", "these", "those", "am", "is", "are", "was", "were",
        "be", "been", "being", "have", "has", "had", "having", "do", "does",
        "did", "doing", "a", "an", "the", "and", "but", "if", "or",
        "because", "as", "until", "while", "of", "at", "by", "for", "with",
        "about", "against", "between", "into", "through", "during", "before",
        "after", "above", "below", "to", "from", "up", "down", "in", "out",
        "off", "over", "under", "again", "further", "then", "once", "here",
        "there", "when", "where", "why", "how", "all", "any", "both", "each",
        "few", "more", "most", "other", "some", "such", "no", "nor", "not",
        "only", "own", "same", "so", "than", "too", "very", "s", "t", "can",
        "will", "just", "don", "should", "now", "on", "thank", "enjoy",
        "please", "may", "even", "forgot", "things", "well", "hey", "much",
        "lets", "look",
        // Contractions without '
        "im", "ive", "id", "Youre", "youd", "youve",
        "hes", "hed", "shes", "shed", "itd", "were", "wed", "weve",
        "theyre", "theyd", "theyve",
        "shouldnt", "couldnt", "musnt", "cant", "wont",
        // Common uppercase words
        "hi", "hello", "url"
    )
    val punctuationSymbols = ".,()!?;:'`´\n<>-’"
    val punctuationRegex = "\\" + punctuationSymbols.split("").mkString("|\\")
    
    // Extract Named Entities from a single text
    def getNEsSingle(text: String): Seq[String] =
      text.replaceAll(punctuationRegex, "").split(" ")
        .filter { word:String => word.length > 1 &&
                  Character.isUpperCase(word.charAt(0)) &&
                  !STOPWORDS.contains(word.toLowerCase) }.toSeq

    def getNEs(textList: Seq[String]): Seq[Seq[String]] = textList.map(getNEsSingle)
    
    //Count and sort Named Entities
    
    def getCounts(result: Seq[Seq[String]]): Seq[Map[String, Double]] = {
      val countsMaps: Seq[Map[String, Double]] = result.map { list =>
        list.foldLeft(Map.empty[String, Double]) {
          (count, word) => count + (word -> (count.getOrElse(word, 0.0) + 1.0))
        }
      } 
      countsMaps.map {
          Map => Map mapValues (_/Map.values.sum) }
    }
    

    def sortNEs(result: Seq[Seq[String]]): List[NERCount] = {
        val counts: Map[String, Double] = getCounts(result).flatten
            .foldLeft(Map.empty[String, Double]) {
            (count, wordCount) => count + (wordCount._1 -> (count.getOrElse(wordCount._1, 0.0) + wordCount._2)) }
        counts.toList.map { case (word, count) => NERCount(word, count) }
        .sortBy(_.count)(Ordering[Double].reverse)
    }
    
}

defined class NERCount
defined class NERModel

In [162]:
val model = new NERModel

model: NERModel = ammonite.$sess.cmd160$Helper$NERModel@5d848e99

### 2.2 Aplicar el "Modelo" a los datos

In [163]:
val resultRss = model.getNEs(listRss)

resultRss: Seq[Seq[String]] = List(
  ArrayBuffer(
    "Chicago",
    "Bears",
    "Anthony",
    "Miller",
    "Germain",
    "Ifedi",
    "Chicago",
    "Bears",
    "Anthony",
    "Miller",
    "Allen",
    "Robinson",
    "Jimmy",
    "Graham",
    "Germain",
    "Ifedi"
  ),
  ArrayBuffer(
    "Justin",
    "Fields",
    "Chicago",
    "Bears",
    "QBs",
    "Conventional",
    "Chicago",
    "Bears",
    "Andy",
    "Dalton",
    "QB",
    "Justin",
    "Fields",
    "Matt",
    "Nagy"
  ),
  ArrayBuffer(
    "Chicago",
    "Cubs",
...

In [165]:
val resultJson = model.getNEs(listJson)

resultJson: Seq[Seq[String]] = List(
  ArrayBuffer(
    "Sunday",
    "Rant/Rage",
    "Note",
    "Join",
    "IRC",
    "Telegram",
    "Sunday",
    "Google/Samsung/HTC",
    "Disagree"
  ),
  ArrayBuffer(
    "Community",
    "Feedback",
    "Poll",
    "February",
    "Participation",
    "NOTES*",
    "Google",
    "Forms",
    "Email",
    "Responses",
    "POLL]",
    "Edit"
  ),
  ArrayBuffer("Google", "I/O", "Keynote"),
  ArrayBuffer("Magisk", "Apple", "Androids"),
  ArrayBuffer("Flexible", "Samsung", "OLED", "SID", "Display", "Week"),
  ArrayBuffer("Android", "Privacy", "Dashboard"),
  ArrayBuffer("New", "Google", "Photos"),
  ArrayBuffer(
    "Samsung",
    "Display",
    "Display",
    "Week",
    "Sfoldable",
    "PC",
    "Slidable",
...

In [164]:
val resultStack = model.getNEs(listStack)

resultStack: Seq[Seq[String]] = List(
  ArrayBuffer("Efficient"),
  ArrayBuffer(),
  ArrayBuffer(),
  ArrayBuffer("Need", "HPC", "MZ", "Azure"),
  ArrayBuffer(),
  ArrayBuffer("Using"),
  ArrayBuffer("SSH2", "Nodejs"),
  ArrayBuffer("PyKinectv2", "Body"),
  ArrayBuffer(),
  ArrayBuffer("Add"),
  ArrayBuffer(
    "Can&#39t",
    "Output",
    "Seaborn",
    "Scatterplot",
    "Tkinter",
    "Frame"
  ),
  ArrayBuffer("Mathrandom"),
  ArrayBuffer("Adding", "Festival", "Software"),
  ArrayBuffer(),
  ArrayBuffer("Storing", "Unity", "WebGL"),
  ArrayBuffer("CSV", "Python", "I&#39d"),
  ArrayBuffer("NetBeans", "JavaFX", "FXMLLoader", "Problem"),
  ArrayBuffer("Replace", "HTML", "Unicode", "Swift", "Unicode"),
  ArrayBuffer("Format", "JSON"),
  ArrayBuffer("Due", "TIME_WAIT"),
  ArrayBuffer("Determine"),
  ArrayBuffer("Order"),
  ArrayBuffer("Type", "Type", "Excel"),
  ArrayBuffer(),
  ArrayBuffer("Route"),
  ArrayBuffer(),
  ArrayBuffer("Scrapy", "Request"),
  ArrayBuffer("Flask"),
  ArrayB

## 3. Contar y ordenar las entidades

Concatenar todas las listas, contar cada Named Entity, y luego ordernar por frecuencia

In [166]:
val stackCounts = model.getCounts(resultStack)

stackCounts: Seq[Map[String, Double]] = List(
  Map("Efficient" -> 1.0),
  Map(),
  Map(),
  Map("Need" -> 0.25, "HPC" -> 0.25, "MZ" -> 0.25, "Azure" -> 0.25),
  Map(),
  Map("Using" -> 1.0),
  Map("SSH2" -> 0.5, "Nodejs" -> 0.5),
  Map("PyKinectv2" -> 0.5, "Body" -> 0.5),
  Map(),
  Map("Add" -> 1.0),
  Map(
    "Frame" -> 0.16666666666666666,
    "Seaborn" -> 0.16666666666666666,
    "Can&#39t" -> 0.16666666666666666,
    "Scatterplot" -> 0.16666666666666666,
    "Output" -> 0.16666666666666666,
    "Tkinter" -> 0.16666666666666666
  ),
  Map("Mathrandom" -> 1.0),
  Map(
    "Adding" -> 0.3333333333333333,
    "Festival" -> 0.3333333333333333,
    "Software" -> 0.3333333333333333
  ),
  Map(),
  Map(
    "Storing" -> 0.3333333333333333,
    "Unity" -> 0.3333333333333333,
    "WebGL" -> 0.3333333333333333
  ),
  Map(
    "CSV" -> 0.3333333333333333,
    "Python" -> 0.3333333333333333,
    "I&#39d" -> 0.3333333333333333
  ),
  Map(
    "NetBeans" -> 0.25,
    "JavaFX" -> 0.25,
...

In [167]:
val rssCounts = model.getCounts(resultRss)

rssCounts: Seq[Map[String, Double]] = List(
  Map(
    "Ifedi" -> 0.125,
    "Anthony" -> 0.125,
    "Germain" -> 0.125,
    "Chicago" -> 0.125,
    "Allen" -> 0.0625,
    "Graham" -> 0.0625,
    "Miller" -> 0.125,
    "Bears" -> 0.125,
    "Robinson" -> 0.0625,
    "Jimmy" -> 0.0625
  ),
  Map(
    "Matt" -> 0.06666666666666667,
    "QBs" -> 0.06666666666666667,
    "QB" -> 0.06666666666666667,
    "Conventional" -> 0.06666666666666667,
    "Chicago" -> 0.13333333333333333,
    "Dalton" -> 0.06666666666666667,
    "Nagy" -> 0.06666666666666667,
    "Andy" -> 0.06666666666666667,
    "Bears" -> 0.13333333333333333,
    "Fields" -> 0.13333333333333333,
    "Justin" -> 0.13333333333333333
  ),
  Map(
    "Lester" -> 0.1111111111111111,
    "Nationals" -> 0.1111111111111111,
    "Washington" -> 0.05555555555555555,
    "Schwarber" -> 0.1111111111111111,
    "Kyle" -> 0.1111111111111111,
    "Chicago" -> 0.1111111111111111,
    "Field" -> 0.05555555555555555,
    "Another" -> 0.05555555555

In [168]:
rssCounts.map { mp => mp.values.sum }

res167: Seq[Double] = List(
  1.0,
  0.9999999999999999,
  1.0000000000000002,
  1.0,
  1.0,
  0.9999999999999998,
  0.9999999999999999,
  0.9999999999999998,
  0.9999999999999998,
  1.0000000000000002,
  1.0,
  1.0,
  1.0,
  1.0,
  0.9999999999999999,
  1.0,
  1.0,
  0.9999999999999998,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.9999999999999996,
  1.0
)

In [169]:
val sortedNEs = model.sortNEs(resultRss)

sortedNEs: List[NERCount] = List(
  NERCount("Chicago", 3.0354384177913585),
  NERCount("Cubs", 1.0075396825396825),
  NERCount("White", 0.7749766573295984),
  NERCount("Sox", 0.6712184873949579),
  NERCount("Bears", 0.583008658008658),
  NERCount("Bulls", 0.5357142857142857),
  NERCount("Field", 0.4913632119514473),
  NERCount("Kyle", 0.42658730158730157),
  NERCount("Photos", 0.39470333587980644),
  NERCount("Fields", 0.3865800865800866),
  NERCount("Justin", 0.3865800865800866),
  NERCount("Abreu", 0.3537581699346405),
  NERCount("Wrigley", 0.346031746031746),
  NERCount("COVID19", 0.34285714285714286),
  NERCount("Tigers", 0.3392857142857143),
  NERCount("Hendricks", 0.3392857142857143),
  NERCount("City", 0.3361928104575163),
  NERCount("Jos\u00e9", 0.31209150326797386),
  NERCount("NBAs", 0.3111111111111111),
  NERCount("Monday", 0.28154761904761905),
  NERCount("Royals", 0.27369281045751637),
  NERCount("Detroit", 0.26785714285714285),
  NERCount("Nationals", 0.25396825396825395

In [170]:
val sortiaDo = model.sortNEs(resultJson)

sortiaDo: List[NERCount] = List(
  NERCount("Android", 2.1337923728813557),
  NERCount("Google", 1.8753177966101695),
  NERCount("Pixel", 0.9929378531073445),
  NERCount("New", 0.7873015873015872),
  NERCount("Pro", 0.723340395480226),
  NERCount("Androids", 0.6002824858757062),
  NERCount("I/O", 0.5502824858757063),
  NERCount("BinaryEye", 0.5),
  NERCount("Microsoft", 0.43333333333333335),
  NERCount("Display", 0.41666666666666663),
  NERCount("Apple", 0.4096045197740113),
  NERCount("RAM", 0.3730579096045198),
  NERCount("Samsung", 0.3584297919250379),
  NERCount("Citra", 0.3333333333333333),
  NERCount("LG", 0.3333333333333333),
  NERCount("Nintendo", 0.3333333333333333),
  NERCount("Privacy", 0.3333333333333333),
  NERCount("Duo", 0.3333333333333333),
  NERCount("Magisk", 0.3333333333333333),
  NERCount("Dex", 0.3333333333333333),
  NERCount("Keynote", 0.3333333333333333),
  NERCount("Photos", 0.3333333333333333),
  NERCount("Holcim", 0.3333333333333333),
  NERCount("Dashboard", 0

In [171]:
val sortedStack = model.sortNEs(resultStack)

sortedStack: List[NERCount] = List(
  NERCount("Using", 1.0),
  NERCount("Efficient", 1.0),
  NERCount("Determine", 1.0),
  NERCount("Add", 1.0),
  NERCount("Route", 1.0),
  NERCount("Flask", 1.0),
  NERCount("Mathrandom", 1.0),
  NERCount("Order", 1.0),
  NERCount("Type", 0.6666666666666666),
  NERCount("Request", 0.5),
  NERCount("Format", 0.5),
  NERCount("SSH2", 0.5),
  NERCount("JSON", 0.5),
  NERCount("Change", 0.5),
  NERCount("Due", 0.5),
  NERCount("Nodejs", 0.5),
  NERCount("TIME_WAIT", 0.5),
  NERCount("Body", 0.5),
  NERCount("PyCharm", 0.5),
  NERCount("PyKinectv2", 0.5),
  NERCount("Scrapy", 0.5),
  NERCount("Unicode", 0.4),
  NERCount("WebGL", 0.3333333333333333),
  NERCount("Software", 0.3333333333333333),
  NERCount("TCP", 0.3333333333333333),
  NERCount("CSV", 0.3333333333333333),
  NERCount("Python", 0.3333333333333333),
  NERCount("Adding", 0.3333333333333333),
  NERCount("Client", 0.3333333333333333),
  NERCount("Excel", 0.3333333333333333),
  NERCount("Socket", 0.

## 4. Modelo "FeedService"

In [202]:
class FeedServices {
    case class Subscription(url: String, parser: DataSource)
    //variable donde se guardan las suscripciones
    var subsList: Seq[Subscription] = Seq()
    
    //funcion para agregar subscripciones a la lista 
    def subscribe[T <: DataSource](template: String, urlParams: Seq[String] = Seq(), parser: T) = {
        val urls: Seq[String] = (if (urlParams.isEmpty) { Seq(template) } else {
            urlParams.map { param => template.format(param) }
        //filtramos para no agregar de nuevo una url que ya estaba en la lista:
        }).filter { url =>
            !(subsList.map(_.url).contains(url))
        }        
        
        val subscriptions: Seq[Subscription] = urls.map {url =>
            Subscription(url, parser)
        }
        
        subsList = subsList ++ subscriptions
    }
    
    //busca todas las entidades en las subscripciones
    //y las añade a una misma lista
    def parseAll(): Seq[String] = {    
        val parsedList = subsList.map { sub =>
            sub.parser.parseResponse(sub.url)
        }
        parsedList.flatten
    }
}

defined class FeedServices

In [232]:
val service = new FeedServices

service: FeedServices = ammonite.$sess.cmd201$Helper$FeedServices@7ffea8f6

In [233]:
//Probamos omitir el parámetro opcional urlParam
service.subscribe(template = "https://www.chicagotribune.com/arcio/rss/category/business/?query=display_date:[now-2d+TO+now]&sort=display_date:desc", urlParams = Seq(), parser = new RssSource())
//Probamos con parámetro
service.subscribe(template = "https://rss.nytimes.com/services/xml/rss/nyt/%s.xml", urlParams = Seq("Business", "Technology"), parser = new RssSource())
//service.subscribe(template = "https://www.reddit.com/r/%s/hot/.json?count=100", urlParams = Seq("Marketing", "Sales", "Entrepreneur", "Startups"), parser = new JsonSource())
//service.subscribe(template = "https://api.stackexchange.com/2.2/questions?order=desc&sort=activity&site=%s", urlParams = Seq("stackoverflow", "serverfault"), parser = new StackQSource())

In [234]:
println(service.subsList)

List(Subscription(https://www.chicagotribune.com/arcio/rss/category/business/?query=display_date:[now-2d+TO+now]&sort=display_date:desc,ammonite.$sess.cmd9$Helper$RssSource@467ef14b), Subscription(https://rss.nytimes.com/services/xml/rss/nyt/Business.xml,ammonite.$sess.cmd9$Helper$RssSource@352dd2db), Subscription(https://rss.nytimes.com/services/xml/rss/nyt/Technology.xml,ammonite.$sess.cmd9$Helper$RssSource@352dd2db))


In [235]:
val texts = service.parseAll()

texts: Seq[String] = List(
  "\u2018Car guy\u2019 Joe Biden took a spin in the new electric-powered Ford F-150 Lightning during a trip to Michigan to sell his $2T infrastructure plan Joe Biden took a spin in the new Ford F-150 Lightning truck during a visit to a Ford safety testing center Tuesday as part of a trip to Michigan. ",
  "COVID-19 has left scars on the job market. Here\u2019s why the damage may be permanent. At least 30% of the U.S. jobs lost to the pandemic aren\u2019t expected to come back \u2014 a sizable proportion of them at employers that require face-to-face contact with consumers: Hotels, restaurants, retailers, entertainment venues. ",
  "In Chicago and across US, downtown stores, restaurants await customers\u2019 return \u201cI definitely think the Loop will return back to normal at some point,\u201d says Teresa Ging of Sugar Bliss Bakery, although she doesn\u2019t expect that to happen before 2022. ",
  "Boeing, FAA grilled by lawmakers over problems with 737 Max,

In [236]:
val model = new NERModel()

model: NERModel = ammonite.$sess.cmd160$Helper$NERModel@406ef726

In [237]:
val res = model.getNEs(texts)

res: Seq[Seq[String]] = List(
  ArrayBuffer(
    "Joe",
    "Biden",
    "Ford",
    "F150",
    "Lightning",
    "Michigan",
    "Joe",
    "Biden",
    "Ford",
    "F150",
    "Lightning",
    "Ford",
    "Tuesday",
    "Michigan"
  ),
  ArrayBuffer("COVID19", "Heres", "US", "Hotels"),
  ArrayBuffer(
    "Chicago",
    "US",
    "Loop",
    "Teresa",
    "Ging",
    "Sugar",
    "Bliss",
    "Bakery"
  ),
  ArrayBuffer(
    "Boeing",
    "FAA",
    "Max",
    "Dreamliner",
    "Two",
    "Congress",
    "Boeing",
    "Federal",
    "Aviation",
    "Administration"
...

In [238]:
val finalRes = model.sortNEs(res)

finalRes: List[NERCount] = List(
  NERCount("China", 1.222843822843823),
  NERCount("Walmart", 1.2222222222222223),
  NERCount("US", 1.2191770647653002),
  NERCount("Biden", 1.1350140056022409),
  NERCount("WarnerMedia", 1.0654382167540062),
  NERCount("AT&T", 1.0071770334928232),
  NERCount("Coinbase", 1.0),
  NERCount("Buying", 1.0),
  NERCount("Apple", 0.9803030303030303),
  NERCount("Discovery", 0.9667710944026733),
  NERCount("President", 0.9598039215686275),
  NERCount("Pipeline", 0.804040404040404),
  NERCount("Colonial", 0.804040404040404),
  NERCount("Amazon", 0.7816239316239315),
  NERCount("Federal", 0.7671568627450981),
  NERCount("Chicago", 0.75503663003663),
  NERCount("Heres", 0.7436507936507937),
  NERCount("Monday", 0.6678571428571427),
  NERCount("Macys", 0.6666666666666666),
  NERCount("CBS", 0.6666666666666666),
  NERCount("COVID19", 0.6428571428571428),
  NERCount("Chinese", 0.6350649350649351),
  NERCount("American", 0.6291666666666667),
  NERCount("Dogecoin", 0.6

In [201]:
println(finalRes)

List(NERCount(Chicago,3.454145299145299), NERCount(County,2.052802181478652), NERCount(Tuesday,1.908567821067821), NERCount(US,1.80413377472201), NERCount(El,1.6225579975579976), NERCount(New,1.5419413919413918), NERCount(Biden,1.4540616246498597), NERCount(Illinois,1.3333333333333333), NERCount(China,1.222843822843823), NERCount(WarnerMedia,1.0654382167540062), NERCount(Lollapalooza,1.0602564102564103), NERCount(Park,1.0444555444555443), NERCount(Es,1.0), NERCount(Coinbase,1.0), NERCount(Buying,1.0), NERCount(Apple,0.9803030303030303), NERCount(Wednesday,0.892156862745098), NERCount(Monday,0.880079365079365), NERCount(Federal,0.8671568627450981), NERCount(President,0.8526610644257704), NERCount(AT&T,0.8405103668261564), NERCount(Pipeline,0.804040404040404), NERCount(Colonial,0.804040404040404), NERCount(Discovery,0.8001044277360067), NERCount(Amazon,0.7816239316239315), NERCount(Washington,0.7598039215686274), NERCount(Trump,0.7556489262371615), NERCount(Police,0.7529151404151404), NE

## 4. Modelo Marcos

1- Hay vario sitios de noticia, por ejemplo tenemos el sitio de noticia del diario la nación, de infobae, de radio mitre.

Modelamos esto con una clase tipo servicio de noticas. Por ejemplo creamos un objeto del sitio LaNacion

*servicioDeNoticias = new ServicioDeNoticas(“LaNacion”);*


2- Cada uno de los sitios de noticia tiene varias formas de acceder a las noticias. Por ejemplo se pueden leer las noticias de la nación en formato xml, en formato url o en texto plano.

Modelamos esto haciendo que el objeto servicioDeNoticia tenga un método obtenerArticulosDesdeUnFormatoXML  que nos devuelve un listado de artículos. Cada articulo tiene  un método título y un método descripción.

*listaDeArticulo = servicioDeNoticia.ObtenerArticulosFormatoXML(url)*



3- Hay dos modelos de clasificación, uno usando un conjunto de reglas y el otro usando redes neuronales. Los modelos tienen un método detectarEntidadesNombradas que reciben una listaDeArticulos y devuelve una lista de entidadesNombradas. 

Modelamos esto usando herencia. Las clases ModeloDeClasificacionUsandoConjuntoDeReglas y  ModeloDeClasificacionUsandoRedesNeuronales heredan de ModeloDeClasificacion. 

La clase ModeloDeClasificacion tiene un metodo virtual DetectarEntidadesNombradas que recibe como parametros una listaDeArticulos y devuelve una listaDeEntidadesNombradas. Las entidades nombradas tienen un método nombre y un método frecuencia.

Modelamos esto de la siguiente manera:

*listaDeEntidadesNombradas = ModeloDeClasificacionUsandoConjuntoDeReglas.DetectarEntidadesNombradas(ListaDeArticulos)*


#Pseudocodigo

In [ ]:
listaDeArticulos = new ListaDeArticulos(Articulos)
listaDeEntidadesNombradas = new ListaDeEntidadesNombradas(EntidadesNombradas)


In [ ]:
servicioDeNoticasLaNacion = new ServicioDeNoticas("LaNacion")
direccionUrlDeLasNoticiasConFormatoXML = "http://unNombreDeDominio.unArchivo.xml"
listaDeArticulo = servicioDeNoticiaLaNacion.ObtenerArticulosDesdeUnFormatoXML(direccionUrlDeLasNoticiasConFormatoXML)
listaDeEntidadesNombradas = ModeloDeClasificacionUsandoConjuntoDeReglas.detectarEntidadesNombradas(ListaDeArticulos)
for entidadNombrada in listaDeEntidadesNombradas:
    print entidadNombrada.Nombre
    print entidadNombrada.Frecuencia


In [ ]:
direccionUrlDeLasNoticiasConFormatoJSON = "http://wxdd.ss.archivo.json"
listaDeArticulos = servicioDeNoticiaLaNacion.ObtenerArticulosDesdeUnFormatoJSON(direccionUrlDeLasNoticiasConFormatoJSON)
listaDeEntidadesNombradas = ModeloDeClasificacionUsandoConjuntoDeReglas.detectarEntidadesNombradas(ListaDeArticulos)
for entidadNombrada in listaDeEntidadesNombradas:
    print entidadNombrada.Nombre
    print entidadNombrada.Frecuencia


In [ ]:
servicioDeNoticasInfobae = new ServicioDeNoticas("Infobae")
listaDeEntidadesNombradasInfobaeTextoPlano = ListaDeEntidadesNombradas(EntidadesNombradas)
direccionUrlDeLasNoticiasConFormatoTextoPlano = "c:/users/documents/archivo.txt"
listaDeArticulos = servicioDeNoticiaInfobae.ObtenerArticulosDesdeUnFormatoTextoPlano(direccionUrlDeLasNoticiasConFormatoTextoPlano)

listaDeEntidadesNombradas = ModeloDeClasificacionUsandoConjuntoDeReglas.detectarEntidadesNombradas(ListaDeArticulos)

for entidadNombrada in listaDeEntidadesNombradas:
    print entidadNombrada.Nombre
    print entidadNombrada.Frecuencia


In [ ]:
Clase: ServicioDeNoticia
Metodos:
obtenerArticulosDesdeUnFormatoXML(direccionUrl):ListaDeArticulos
obtenerArticulosDesdeUnFormatoJSON(direccionUrl):ListaDeArticulos

ListaDeArticulos
Clase:Articulo
Metodos:
  Titulo
  Descripcion


ModeloDeClasificacionUsandoRedesNeuronales:ModeloDeClasificacion
ModeloDeClasificacionUsandoConjuntoDeReglas:ModeloDeClasificacion
detectarEntidadesNombradas(ListaDeArticulos):EntidadesNombradas

ListaDeEntidadesNombradas
Clase: EntidadesNombradas
Metodos:
  Nombre
  Frecuencia
